<a href="https://colab.research.google.com/github/levimcclenny/notebooks/blob/main/Notebook3_AC_Parameter_estimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AC Parameter Estimation
In this example, we seek to estimate the parameters of the Allen-Cahn PDE system used in the last notebook. We can perform this estimation in *tensorDiffEq* using the DiscoveryModel()| 

First, install the package

In [1]:
!pip install tensordiffeq

Imports as usual, make sure to import the DiscoveryModel class!

In [2]:
import numpy as np
import tensorflow as tf
import scipy.io
import tensordiffeq as tdq
from tensordiffeq.models import DiscoveryModel

Create a list of tf.Variables for the parameters to be estimated.

In [3]:
params = [tf.Variable(0.0, dtype=tf.float32), tf.Variable(0.0, dtype=tf.float32)]

Generate the strong-form PDE, replacing the parameters we are estimating with the tf.Variables defined in the params variable. In this case, var ends up being the params list described in the cell above

In [4]:
# Define f_model, note the `vars` argument. Inputs must follow this order!
def f_model(u_model, var, x, t):
    u = u_model(tf.concat([x, t], 1))
    u_x = tf.gradients(u, x)
    u_xx = tf.gradients(u_x, x)
    u_t = tf.gradients(u, t)
    c1 = var[0]  # tunable param 1
    c2 = var[1]  # tunable param 2
    f_u = u_t - c1 * u_xx + c2 * u * u * u - c2 * u
    return f_u

Import the Allen-Cahn data

In [5]:
# Get AC data from repo, requires the ?raw=true so wget doesnt download the webpage instead of the data!
!wget --no-check-certificate https://github.com/tensordiffeq/TensorDiffEq/blob/main/examples/AC.mat?raw=true \
    -O AC.mat

--2021-05-06 05:00:57--  https://github.com/tensordiffeq/TensorDiffEq/blob/main/examples/AC.mat?raw=true
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/tensordiffeq/TensorDiffEq/raw/main/examples/AC.mat [following]
--2021-05-06 05:00:57--  https://github.com/tensordiffeq/TensorDiffEq/raw/main/examples/AC.mat
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/tensordiffeq/TensorDiffEq/main/examples/AC.mat [following]
--2021-05-06 05:00:57--  https://raw.githubusercontent.com/tensordiffeq/TensorDiffEq/main/examples/AC.mat
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTT

Import and reformat data for use in tdq

In [6]:
data = scipy.io.loadmat('AC.mat')

t = data['tt'].flatten()[:, None]
x = data['x'].flatten()[:, None]
Exact = data['uu']
Exact_u = np.real(Exact)

Create input data for the target data in AC.mat and generate a list of input data. In this case, the targets are stored in X_star, and the inputs are stored in X

In [7]:
# generate all combinations of x and t
X, T = np.meshgrid(x, t)

X_star = np.hstack((X.flatten()[:, None], T.flatten()[:, None]))
u_star = Exact_u.T.flatten()[:, None]

x = X_star[:, 0:1]
t = X_star[:, 1:2]

# append to a list for input to model.fit
X = [x, t]

Generate col_weights (if using SA solving) and layer sizes for the approimation neural network

In [8]:
# define col_weights for SA discovery model, can be removed
col_weights = tf.Variable(tf.random.uniform([np.shape(x)[0], 1]))

# define MLP depth and layer width
layer_sizes = [2, 128, 128, 128, 128, 1]

Import the model and compile

In [9]:
model = DiscoveryModel()
model.compile(layer_sizes, f_model, X, u_star, params) 

This is optional - define a different optimizer from the Keras suite of optimizers

In [10]:
# an example as to how one could modify an optimizer, in this case the col_weights optimizer
model.tf_optimizer_weights = tf.keras.optimizers.Adam(lr=0.005,
                                                      beta_1=.95)



Train the model with 10k Adam iterations

In [11]:
# train loop
model.fit(tf_iter=10000)


[<tf.Variable 'Variable:0' shape=() dtype=float32>, <tf.Variable 'Variable:0' shape=() dtype=float32>]
[<tf.Variable 'Variable:0' shape=() dtype=float32>, <tf.Variable 'Variable:0' shape=() dtype=float32>]
It: 0, Time: 2.16
loss_value: 0.5344747304916382
vars estimate(s): [-0.0049884045, 0.004997983]
It: 100, Time: 12.81
loss_value: 0.23482635617256165
vars estimate(s): [0.0076214774, -0.055614177]
It: 200, Time: 13.11
loss_value: 0.1246507316827774
vars estimate(s): [0.0064566415, -0.0753046]
It: 300, Time: 13.19
loss_value: 0.09062732011079788
vars estimate(s): [0.0025977604, -0.08056378]
It: 400, Time: 12.91
loss_value: 0.0749722570180893
vars estimate(s): [0.0016510101, -0.07313789]
It: 500, Time: 12.81
loss_value: 0.06650445610284805
vars estimate(s): [-0.00091971294, -0.05759476]
It: 600, Time: 12.81
loss_value: 0.06338178366422653
vars estimate(s): [-0.0009755291, -0.033789974]
It: 700, Time: 12.89
loss_value: 0.060310978442430496
vars estimate(s): [-0.0013915153, 0.0014047724]
